<a href="https://colab.research.google.com/github/xCosmicx/ATA/blob/main/week4/3.fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning

Another widely used transfer learning technique is _fine-tuning_. 
Fine-tuning involves unfreezing a few of the top layers 
of a frozen model base used for feature extraction, and jointly training both the newly added part of the model (in our case, the 
fully-connected classifier) and these unfrozen top layers. This is called "fine-tuning" because it slightly adjusts the more abstract 
representations of the model being reused, in order to make them more relevant for the problem at hand.

![fine-tuning VGG16](https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/it3103/resources/vgg16_fine_tuning.png)

In [1]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

It was necessary to freeze the convolution base of VGG16 in order to be able to train a randomly initialized 
classifier on top. For the same reason, it is only possible to fine-tune the top layers of the convolutional base once the classifier on 
top has already been trained. If the classified wasn't already trained, then the error signal propagating through the network during 
training would be too large, and the representations previously learned by the layers being fine-tuned would be destroyed. Thus the steps 
for fine-tuning a network are as follow:

1. Add your custom network on top of an already trained base network.
2. Freeze the base network.
3. Train the part you added.
4. Unfreeze some layers in the base network.
5. Jointly train both these layers and the part you added.


In [2]:
img_height, img_width = 128, 128

# Load the pre-trained model 
base_model = keras.applications.VGG16(input_shape=(img_height, img_width) + (3,),
                                         include_top=False,
                                         weights='imagenet')

preprocess_input_fn = keras.applications.vgg16.preprocess_input

# freeze the base layer 
base_model.trainable = False

# Add input layer 
inputs = layers.Input(shape=(img_height, img_width, 3))
# Add preprocessing layer
x = preprocess_input_fn(inputs)
# Add the base, set training to false to freeze the convolutional base
x = base_model(x)
# Add our classification head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(rate=0.5)(x)
x = layers.Dense(units=512, activation="relu")(x)
x = layers.Dropout(rate=0.5)(x)
outputs = layers.Dense(units=1, activation="sigmoid")(x)

model = keras.models.Model(inputs=[inputs], outputs=[outputs])

base_learning_rate = 0.001

model.compile(loss="binary_crossentropy", 
                  optimizer=keras.optimizers.Adam(learning_rate=base_learning_rate), 
                  metrics=["accuracy"])


58900480/58889256 [==============================] - 1s 0us/step


Let's confirm all the layers of convolutional base are frozen. 

In [3]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


Let's print out the model summary and see how many trainable weights. We can see that we only 263,169 trainable weights (parameters), coming from the classification head that put on top of the convolutional base. (For comparison, a VGG16 has total of 14,714,688 weights).

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0     

## Creating Datasets

We will setup our training and validation dataset as we did in earlier exercise.

In [5]:
dataset_URL = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/cats_and_dogs_subset.tar.gz'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs_subset.tar.gz', origin=dataset_URL, extract=True, cache_dir='.')
dataset_dir = os.path.join(os.path.dirname(path_to_zip), "cats_and_dogs_subset")

67051520/67041740 [==============================] - 5s 0us/step


In [6]:
batch_size = 32
image_size = (img_height, img_width)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.
Found 3000 files belonging to 2 classes.
Using 600 files for validation.


## Train the classification head 

We will go ahead and train our classification head.

In [7]:
# create model checkpoint callback to save the best model checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_checkpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(train_ds, validation_data=val_ds, 
          epochs=30, callbacks=[model_checkpoint_callback])

Epoch 1/30
75/75 [==============================] - 19s 87ms/step - loss: 1.8639 - accuracy: 0.8517 - val_loss: 0.4198 - val_accuracy: 0.9417
Epoch 2/30
75/75 [==============================] - 6s 73ms/step - loss: 0.8488 - accuracy: 0.9104 - val_loss: 0.4033 - val_accuracy: 0.9450
Epoch 3/30
75/75 [==============================] - 5s 70ms/step - loss: 0.7672 - accuracy: 0.9175 - val_loss: 0.3106 - val_accuracy: 0.9367
Epoch 4/30
75/75 [==============================] - 6s 71ms/step - loss: 0.5993 - accuracy: 0.9183 - val_loss: 0.2460 - val_accuracy: 0.9483
Epoch 5/30
75/75 [==============================] - 5s 70ms/step - loss: 0.4176 - accuracy: 0.9258 - val_loss: 0.2281 - val_accuracy: 0.9400
Epoch 6/30
75/75 [==============================] - 5s 70ms/step - loss: 0.3037 - accuracy: 0.9308 - val_loss: 0.2300 - val_accuracy: 0.9417
Epoch 7/30
75/75 [==============================] - 6s 71ms/step - loss: 0.3084 - accuracy: 0.9304 - val_loss: 0.1696 - val_accuracy: 0.9433
Epoch 8/30
7

Now we have our classification layers trained, let's start to unfreeze some top layers of the convolutional base to fine tune the weights. 
We will fine-tune the last 3 convolutional layers, which means that all layers up until `block4_pool` should be frozen, and the layers 
`block5_conv1`, `block5_conv2` and `block5_conv3` should be trainable.

Why not fine-tune more layers? Why not fine-tune the entire convolutional base? We could. However, we need to consider that:

* Earlier layers in the convolutional base encode more generic, reusable features, while layers higher up encode more specialized features. It is 
more useful to fine-tune the more specialized features, as these are the ones that need to be repurposed on our new problem. There would 
be fast-decreasing returns in fine-tuning lower layers.
* The more parameters we are training, the more we are at risk of overfitting. The convolutional base has 15M parameters, so it would be 
risky to attempt to train it on our small dataset.

Thus, in our situation, it is a good strategy to only fine-tune the top 2 to 3 layers in the convolutional base.

Let's set this up, we will unfreeze our `base_model`, 
and then freeze individual layers inside of it, except the last 3 layers. 

Do a model ``summary()`` and you will see now that the number of trainable weights are now 7,079,424 (around 7 millions), much less than previously, because all the layers are frozen except the last 3 layers.

In [8]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

Let us examine model summary again. We can see now that we have more trainable weights 7,342,593 compared to previously 263,169.

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0     

As you are training a much larger model and want to readapt the pretrained weights, it is important to use a lower learning rate at this stage as we do not want to make too drastic changes to the weights in the convolutional layers under fine-tuning.

In [10]:
finetune_learning_rate = base_learning_rate / 10.

model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=finetune_learning_rate),
              metrics=["accuracy"])

model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    callbacks=[model_checkpoint_callback])

Epoch 1/15
75/75 [==============================] - 8s 80ms/step - loss: 0.2225 - accuracy: 0.9262 - val_loss: 0.1342 - val_accuracy: 0.9533
Epoch 2/15
75/75 [==============================] - 6s 78ms/step - loss: 0.1009 - accuracy: 0.9658 - val_loss: 0.1213 - val_accuracy: 0.9467
Epoch 3/15
75/75 [==============================] - 6s 78ms/step - loss: 0.0592 - accuracy: 0.9787 - val_loss: 0.1549 - val_accuracy: 0.9383
Epoch 4/15
75/75 [==============================] - 6s 82ms/step - loss: 0.0312 - accuracy: 0.9892 - val_loss: 0.1158 - val_accuracy: 0.9583
Epoch 5/15
75/75 [==============================] - 6s 78ms/step - loss: 0.0330 - accuracy: 0.9858 - val_loss: 0.1672 - val_accuracy: 0.9483
Epoch 6/15
75/75 [==============================] - 6s 79ms/step - loss: 0.0292 - accuracy: 0.9908 - val_loss: 0.1589 - val_accuracy: 0.9517
Epoch 7/15
75/75 [==============================] - 6s 78ms/step - loss: 0.0112 - accuracy: 0.9962 - val_loss: 0.1842 - val_accuracy: 0.9450
Epoch 8/15
75

In [11]:
model.load_weights('best_checkpoint')
model.evaluate(val_ds)

19/19 [==============================] - 1s 56ms/step - loss: 0.1158 - accuracy: 0.9583


[0.11575710028409958, 0.9583333134651184]

**Exercise:**

1. Is our fine-tuned model performing better or worse? 
2. Try to unfreeze less/more layers and see if the model performance improves.
